### Vitoria Moreno-Costa's Notebook
DSI Group Project

Team 6: Emily Fuller, Karthik Nambiar, Vitoria Moreno-Costa, and David Wagenhurst

#### Imports

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests

### Read in datasets, reduce content to Texas only, and export to cleaned_datasets

#### US Petroleum Refineries

In [65]:
# df_refineries = pd.read_csv('datasets/us-petroleum-refineries.csv')

# df_refineries.columns = df_refineries.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

# df_texas_refineries = df_refineries[df_refineries['state'] == 'Texas']

# df_texas_refineries.loc[:,'state'] = 'TX'

# df_texas_refineries.to_csv('cleaned_datasets/texas_petroleum_refineries.csv',index=False)

#### EPA 2017 AirToxScreen: Cancer Risk by Source Group

In [66]:
# df_epa_cancer_by_source = pd.read_excel('datasets/national_cancerrisk_by_tract_srcgrp.xlsx')

# df_epa_cancer_by_source.columns = df_epa_cancer_by_source.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

# df_epa_cancer_by_source_TX = df_epa_cancer_by_source[df_epa_cancer_by_source['state'] == 'TX']

# df_epa_cancer_by_source_TX.to_csv('cleaned_datasets/texas_cancerrisk_by_srcgrp.csv',index=False)

#### EPA 2017 AirToxScreen: Cancer Risk by Pollutant

In [79]:
# df_epa_cancer_by_pollutant = pd.read_excel('datasets/national_cancerrisk_by_tract_poll.xlsx',)

# df_epa_cancer_by_pollutant.columns = df_epa_cancer_by_pollutant.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

# df_epa_cancer_by_pollutant_TX = df_epa_cancer_by_pollutant[df_epa_cancer_by_pollutant['state'] == 'TX']

# df_epa_cancer_by_pollutant_TX.to_csv('cleaned_datasets/texas_cancerrisk_by_tract_poll.csv', index=False)

#### EPA 2017 AirToxScreen: Emissions by Source

In [125]:
# df_epa_emissions_by_source = pd.read_excel('datasets/point_fac_2017_emissions.xlsx')

# df_epa_emissions_by_source.columns = df_epa_emissions_by_source.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

# df_epa_emissions_by_source_TX = df_epa_emissions_by_source[df_epa_emissions_by_source['state'] == 'Texas']

# df_epa_emissions_by_source_TX.loc[:,'state'] = 'TX'

# # Drop non-numerical values for facility-id
# df_epa_emissions_by_source_TX['facility_id'] = pd.to_numeric(df_epa_emissions_by_source_TX['facility_id'], errors='coerce')

# df_epa_emissions_by_source_TX.dropna(inplace=True)

# df_epa_emissions_by_source_TX.to_csv('cleaned_datasets/texas_point_fac_2017_emissions.csv', index=False)

#### EPA 2017 AirToxScreen: All Hazard Indices by Source

In [81]:
# df_epa_all_hazard_index = pd.read_excel('datasets/national_allhi_by_tract.xlsx')

# df_epa_all_hazard_index.columns = df_epa_all_hazard_index.columns.astype(str).str.strip().str.lower().str.replace(' ','_')

# df_epa_all_hazard_index_TX = df_epa_all_hazard_index[df_epa_all_hazard_index['state'] == 'TX']

# df_epa_all_hazard_index_TX.to_csv('cleaned_datasets/texas_allhi_by_tract.csv', index=False)

### Combine Datasets as possible

In [170]:
df_refineries = pd.read_csv('cleaned_datasets/texas_petroleum_refineries.csv')

In [126]:
df_emissions = pd.read_csv('cleaned_datasets/texas_point_fac_2017_emissions.csv')

In [127]:
df_emissions.head(2)

,facility_id,facility_name,fips,county,state,latitude,longitude,"1,1,2,2-tetrachloroethane_(year_2017_tons)","1,1,2-trichloroethane_(year_2017_tons)","1,1-dimethyl_hydrazine_(year_2017_tons)",...,m-cresol_(year_2017_tons),m-xylene_(year_2017_tons),o-anisidine_(year_2017_tons),o-cresol_(year_2017_tons),o-toluidine_(year_2017_tons),o-xylene_(year_2017_tons),p-cresol_(year_2017_tons),p-dioxane_(year_2017_tons),p-phenylenediamine_(year_2017_tons),p-xylene_(year_2017_tons)
0,10677711.0,TYSON FRESH MEATS,48375,Potter Co,TX,35.256694,-101.646269,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10677811.0,GREY RANCH COMPRESSOR STATION,48371,Pecos Co,TX,30.554169,-102.443892,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [175]:
df_emissions.columns

Index(['facility_id', 'facility_name', 'fips', 'county', 'state', 'latitude',
       'longitude', '1,1,2,2-tetrachloroethane_(year_2017_tons)',
       '1,1,2-trichloroethane_(year_2017_tons)',
       '1,1-dimethyl_hydrazine_(year_2017_tons)',
       ...
       'm-cresol_(year_2017_tons)', 'm-xylene_(year_2017_tons)',
       'o-anisidine_(year_2017_tons)', 'o-cresol_(year_2017_tons)',
       'o-toluidine_(year_2017_tons)', 'o-xylene_(year_2017_tons)',
       'p-cresol_(year_2017_tons)', 'p-dioxane_(year_2017_tons)',
       'p-phenylenediamine_(year_2017_tons)', 'p-xylene_(year_2017_tons)'],
      dtype='object', length=272)

#### Use census API to identify FIPs for each refinery

In [171]:
def coord_to_tract(lat, long):
    """ This function identifies the FIPs census tract number for a given latitude and longitude using the 2010 census.
    
    args:
        lat (float): Latitude
        long (float): Longitude
        
    return:
        tract (int): census tract number, which is the first 11 digits of the census block FIPs
    """
    url = 'https://geo.fcc.gov/api/census/block/find'
    
    params = {
        'latitude': lat,
        'longitude': long,
        'censusYear': 2010,
        'format':'json'
    }
    res = requests.get(url,params)
    if res.status_code == 200:
        fips = res.json()['Block']['FIPS']
        return fips[0:11]
    else:
        print('request_failed')

In [172]:
refinery_tract = []
for refinery in df_refineries.index:
    coords = df_refineries.loc[refinery,['latitude','longitude']]
    refinery_tract.append(coord_to_tract(coords[0],coords[1]))

In [173]:
df_refineries['tract'] = refinery_tract

In [181]:
# save to replace csv
df_refineries.to_csv('cleaned_datasets/texas_petroleum_refineries.csv',index=False)

#### Now do the same for the emissions data

In [176]:
emission_tract = []
for source in df_emissions.index:
    coords = df_emissions.loc[source,['latitude','longitude']]
    emission_tract.append(coord_to_tract(coords[0],coords[1]))

In [177]:
df_emissions['tract'] = emission_tract

In [182]:
# save to replace csv
df_emissions.to_csv('cleaned_datasets/texas_point_fac_2017_emissions.csv',index=False)